# Tutorial for Graph Generation

In this hand-on code tutorial, we will show how to generate graphs using our DIG library. Specifically, we show how to use the implemented GraphEBM and GraphDF methods to create molecular graph generators with deep generative models.

## Graph Generation

In drug discovery and chemical science, a fundamental problem is to design and synthesize novel molecules with some desirable properties (e.g. high drug-likeness). This problem remains to be very challenging, because the space of molecules is discrete and very huge. A promising solution is to construct a graph generator which can automatically generate novel molecular graphs. Recently, many approaches are proposed for molecular graph generation, such as JT-VAE, GCPN, GraphAF, GraphEBM, and GraphDF.

To generate molecular graphs, we first need to decide what is generated to form a molecular graph. Generally, the following three graph formation methods are most widely used in existing molecular graph generation approaches:

* Tree-based method. The tree structure of a molecule is firstly generated, where the nodes of tree represent a motif or subgraph of the molecular graph, e.g., an aromatic ring. Then, for any two connected subgraphs in the tree, the binding points between them are decided, and a molecular graph is finally formed by binding all subgraphs. An example of this method is JT-VAE.
* One-shot method. The molecular graph is formed by explicitly generating its node type matrix and adjacency tensor. Examples of this method are GraphVAE and GraphEBM.
* Sequential method. The molecular graph is formed step by step, where only one node or one edge is generated in each step. Examples of this method are GCPN, GraphAF and GraphDF.

After the molecular graph formation method is determined, we can use any deep generative model (e.g. VAE, GAN, and flow) to construct a graph generator, in which latent variables are mapped to the generation targets by the model.

In this tutorial, we will use [GraphEBM](https://arxiv.org/abs/2102.00546) and [GraphDF](https://arxiv.org/abs/2102.01189) as examples to show how to use one-shot and sequential methods in the molecular graph generation problem, separately. Specifically, we will show how to use our implemented APIs to quickly running training and generation on the ZINC250k dataset with code examples.

## GraphEBM

GraphEBM is a one-shot and permutation-invariant molecular graph generation method based on energy-based models. Next, we show how to develop the pipeline of training on the ZINC250k dataset, generating molecular graphs, and evaluating the generation performance with GraphEBM.

In [1]:
import os
import torch
from torch_geometric.loader import DenseDataLoader
from rdkit import RDLogger

from dig.ggraph.dataset import QM9, ZINC250k
from dig.ggraph.method import GraphEBM
from dig.ggraph.evaluation import RandGenEvaluator
device = torch.device('cuda:1')

First, we set up the dataset class and loader for the ZINC250k dataset.

In [2]:
dataset = ZINC250k(one_shot=True, root='./')
splits = dataset.get_split_idx()
train_set = dataset[splits['train_idx']]
train_dataloader = DenseDataLoader(train_set, batch_size=128, shuffle=True, num_workers=0)

making raw files: ./raw


Processing...
making processed files: ./zinc250k_property/processed_oneshot
Done!


Next, we create the GraphEBM based molecular graph generator by defining an object `graphebm` as an instance of class `GraphEBM`, and start training the GraphEBM model on the ZINC250k dataset. The checkpoint of each epoch is saved under the folder 'checkpoints'.

In [3]:
graphebm = GraphEBM(n_atom=38, n_atom_type=10, n_edge_type=4, hidden=64, device=device)

In [ ]:
graphebm.train_rand_gen(train_dataloader, lr=1e-4, wd=0, max_epochs=20, c=0, ld_step=150, ld_noise=0.005, ld_step_size=30, clamp=True, alpha=1, save_interval=1, save_dir='./checkpoints')

When the training completes, we can use the trained generator to generate molecular graphs. The generated molecules are represented by a list of `rdkit.Chem.Mol` objects.

**Skip training**: You can also download and use [our trained models](https://github.com/divelab/DIG_storage/blob/main/ggraph/GraphEBM/GraphEBM_zinc250k_uncond.pt) on the ZINC250k dataset. Note: We found that we might have an error about loading the trained models if we download them with `wget`. If you have the same error, please download the models manually.

In [4]:
### Ignore info output by RDKit
RDLogger.DisableLog('rdApp.error') 
RDLogger.DisableLog('rdApp.warning')

atomic_num_list = dataset.atom_list  # [6, 7, 8, 9, 15, 16, 17, 35, 53, 0] for ZINC250k
gen_mols = graphebm.run_rand_gen(checkpoint_path='./GraphEBM_zinc250k_uncond.pt', n_samples=10000, c=0, ld_step=150, ld_noise=0.005, ld_step_size=30, clamp=True, atomic_num_list=atomic_num_list)

Loading paramaters from ./GraphEBM_zinc250k_uncond.pt
Initializing samples...
Generating samples...


Finally, we call the evaluator to evaluate the generated molecules by the validity ratio, the uniqueness ratio, and the novelty ratio.

In [6]:
train_smiles = [data.smile for data in dataset[splits['train_idx']]]
res_dict = {'mols':gen_mols, 'train_smiles': train_smiles}
evaluator = RandGenEvaluator()
results = evaluator.eval(res_dict)
print(results)

Valid Ratio: 10000/10000 = 100.00%
Unique Ratio: 9805/10000 = 98.05%
Novel Ratio: 10000/10000 = 100.00%
{'valid_ratio': 100.0, 'unique_ratio': 98.05, 'novel_ratio': 100.0}


## GraphDF

GraphDF is a sequential molecular graph generation methods based on discrete flow models. Next, we show how to develop the pipeline of training on the ZINC250k dataset, generating molecular graphs, and evaluating the generation performance with GraphDF.

First, we set up the dataset class and loader for the ZINC250k dataset. Note that when setting up dataset for sequential generation, the `one_shot` parameter of the `ZINC250k` class needs to be `False`.

In [8]:
import json
from dig.ggraph.dataset import ZINC250k
from torch_geometric.loader import DenseDataLoader
conf = json.load(open('../../examples/ggraph/GraphDF/config/rand_gen_zinc250k_config_dict.json'))
dataset = ZINC250k(one_shot=False, use_aug=True, root="./zinc250k_seq")
loader = DenseDataLoader(dataset, batch_size=conf['batch_size'], shuffle=True)

making raw files: zinc250k_seq/raw


Processing...
making processed files: zinc250k_seq/zinc250k_property/processed
Done!


Next, we create the GraphDF based molecular graph generator by defining an object `runner` as an instance of class `GraphDF`, and start training the GraphDF model on the ZINC250k dataset. The checkpoints of each epoch is saved under the folder 'rand_gen_zinc250k'.

In [9]:
from dig.ggraph.method import GraphDF
runner = GraphDF()
lr = 0.001
wd = 0
max_epochs = 10
save_interval = 1
save_dir = 'rand_gen_zinc250k'
runner.train_rand_gen(loader=loader, lr=lr, wd=wd, max_epochs=max_epochs,
    model_conf_dict=conf['model'], save_interval=save_interval, save_dir=save_dir)

When the training completes, we can use the trained generator to generate molecular graphs. The generated molecules are represented by a list of `rdkit.Chem.Mol` objects.

**Skip training**: You can also download and use [our trained models](https://github.com/divelab/DIG_storage/blob/main/ggraph/GraphDF/saved_ckpts/rand_gen/rand_gen_zinc250k.pth) on the ZINC250k dataset.

In [10]:
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')
ckpt_path = './rand_gen_zinc250k.pth'
n_mols = 1000
mols, pure_valids = runner.run_rand_gen(model_conf_dict=conf['model'], checkpoint_path=ckpt_path,
    n_mols=n_mols, atomic_num_list=conf['atom_list'])

Finally, we call the evaluator to evaluate the generated molecules by the validity ratio, the uniqueness ratio, and the novelty ratio. Since the bond valency check is used in the sequential generation process of GraphDF, we use the validity ratio without valency check as an additional evaluation metric.

In [11]:
from dig.ggraph.evaluation import RandGenEvaluator
evaluator = RandGenEvaluator()
smiles = [data.smile for data in dataset]
input_dict = {'mols': mols, 'train_smiles': smiles}
results = evaluator.eval(input_dict)
print("Valid Ratio without valency check: {:.2f}%".format(sum(pure_valids) / n_mols * 100))
print(results)

Valid Ratio: 10000/10000 = 100.00%
Unique Ratio: 9805/10000 = 99.86%
Novel Ratio: 10000/10000 = 100.00%
Valid Ratio without valency check: 89.54%
{'valid_ratio': 100.0, 'unique_ratio': 98.05, 'novel_ratio': 100.0}
